# Calibrating the HBV model

## Setting up the python kernel

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import ewatercycle
import ewatercycle.models
import ewatercycle.forcing

from rich import print
from tqdm import tqdm
from pathlib import Path
from ewatercycle_DA import DA
from datetime import datetime
from scipy.stats import qmc
from ipywidgets import IntProgress

## Loading in observed discharge data

In [2]:
# Loading in discharge data from Katima Mulilo
katima = (pd.read_csv('Katima_Mulilo/1291100_Q_Day.txt', skiprows=35, delimiter=';', header=1)
    .drop(columns=['hh:mm'])
    .rename(columns={"YYYY-MM-DD": "Date", " Value": "Discharge [m^3/s]"}))

# Creating a dataframe with accessible dates and no invalid data 
katima['Date'] = pd.to_datetime(katima['Date'])
katima['Discharge [m^3/s]'] = katima['Discharge [m^3/s]'].replace(-999.000, np.nan)

# Defining the data period
start_calibration = '1986-01-01'
end_calibration = '2019-12-31'

# Reducing the dataframe to the calibration period
km = katima[(katima['Date'] >= start_calibration) & (katima['Date'] <= end_calibration)]

# Creating a dataframe of the flow data 
flow = pd.DataFrame(data=km['Discharge [m^3/s]'].values, index=km['Date'], columns=['Q'])

1

## Generating (or loading in) ERA5 forcings 

In [3]:
#Defining the location of the shape file to be used
km_shp = Path.home() / "BEP-Zoe/book/thesis_projects/BSc/2025_Q4_ZoeLucius_CEG" /  "km_ERA5" / "km.shp"

#Defining the forcings directory
ERA5_folder = Path.home() / "ERA5"
ERA5_folder.mkdir(exist_ok=True)

# HBV model path 
HBV_model_path = Path.home() / "HBV_model"

# OPTION 1: Generating ERA5 forcings
# ERA5_forcing = ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(
#    dataset="ERA5",
#    start_time= data_start,
#    end_time= data_end,
#    shape=km_shp,
#    directory=ERA5_folder)

# OPTION 2: Generating ERA5 forcings (For when ERA5 forcing has already been made before and exists in the directory)
load_location = ERA5_folder / "work" / "diagnostic" / "script" 
ERA5_forcing = ewatercycle.forcing.sources["LumpedMakkinkForcing"].load(directory=load_location)

## Defining functions to determine the model of best fit

### Root Mean Square Error

In [33]:
def RMSE(output, observed, start, end):
     # Making sure the given dates are in the correct format
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    
    # Making sure the date of the modelled output and onserved input are in the correct format
    output.index = pd.to_datetime(output.index)
    observed.index = pd.to_datetime(observed.index)

    # Combine the model output and the observation into one data frame 
    hydro_data = pd.concat([output.reindex(observed.index, method='ffill'), observed], axis=1, keys=['model', 'observation'])
    hydro_data = hydro_data.dropna()
    
    # Making sure to only take the calibration period
    hydro_data = hydro_data[(hydro_data.index > start) & (hydro_data.index < end)]
    
    # Calculate the absolute square difference
    squarediff = (hydro_data['model'] - hydro_data['observation']) ** 2
    rootMeanSquareDiff = np.sqrt(np.mean(squarediff))
    
    return rootMeanSquareDiff


### Kling Gupta Efficiency

In [58]:
def KGE(output, observed, start, end):
    # Making sure the given dates are in the correct format
    start = pd.to_datetime(start).normalize()
    end = pd.to_datetime(end).normalize()
    
    # Making sure the date of the modelled output and onserved input are in the correct format
    output.index = pd.to_datetime(output.index).normalize()
    observed.index = pd.to_datetime(observed.index).normalize()
    
    # Making sure to only take the calibration period
    ouput = output[(output.index > start) & (output.index < end)]
    observed = observed[(observed.index > start) & (observed.index < end)]

    common_index = output.index.intersection(observed.index)
    output = output.loc[common_index]
    observed = observed.loc[common_index]

    output = output.dropna()
    observed = observed.dropna()

    if len(output) == 0 or len(observed) == 0:
        return np.nan

    # Determining coefficients for the kge
    r = np.corrcoef(observed, output)[0, 1]
    beta = np.mean(output) / np.mean(observed)
    gamma = (np.std(output) / np.mean(output)) / (np.std(observed) / np.mean(observed))
    
    kge = 1 - np.sqrt((r - 1)**2 + (beta - 1)**2 + (gamma - 1)**2)
    
    return kge

def KGE2(output, observed, start, end):
    # Normalize datetime indices
    output.index = pd.to_datetime(output.index).normalize()
    observed.index = pd.to_datetime(observed.index).normalize()

    # Normalize start and end dates
    start = pd.to_datetime(start).normalize()
    end = pd.to_datetime(end).normalize()

    # Subset time range
    output = output[(output.index >= start) & (output.index <= end)]
    observed = observed[(observed.index >= start) & (observed.index <= end)]

    # Combine and align
    hydro_data = pd.concat([output, observed], axis=1)
    hydro_data.columns = ['output', 'observed']
    hydro_data = hydro_data.dropna()

    if len(hydro_data) == 0:
        return np.nan  # avoid error

    # Compute KGE components
    r = np.corrcoef(hydro_data['observed'], hydro_data['output'])[0, 1]
    beta = np.mean(hydro_data['output']) / np.mean(hydro_data['observed'])
    gamma = (np.std(hydro_data['output']) / np.mean(hydro_data['output'])) / \
            (np.std(hydro_data['observed']) / np.mean(hydro_data['observed']))

    kge = 1 - np.sqrt((r - 1)**2 + (beta - 1)**2 + (gamma - 1)**2)
    return kge


## Start calibration

In [35]:
# Set the number of HBV model ensembles you will test against the observed discharge
N = 150

In [36]:
# Define the initial state parameters (Si,  Su, Sf, Ss, Sp)
s_0 = np.array([0,  100,  0,  5,  0])

# Define parameters and their corresponding boundary values 
param_names = ["Imax", "Ce", "Sumax", "Beta", "Pmax", "Tlag", "Kf", "Ks", "FM"]
param_mins = np.array([0, 0.2, 40, 0.5, 0.001, 1, 0.01, 0.0001, 0.01])
param_maxs = np.array([8, 1, 800, 4, 0.3, 10, 0.1, 0.01, 1])

#Fill the parameters array with random values between each minimum and maximum 
sampler = qmc.LatinHypercube(d=len(param_names))
sample = sampler.random(n=N)
parameters = qmc.scale(sample, param_mins, param_maxs)

In [37]:
# Observe the randomly chosen variables for each parameter to confirm their boundaries are adhered to
print(list(zip(param_names, np.round(parameters[0], decimals=3))))

[
    ('Imax', 4.819),
    ('Ce', 0.317),
    ('Sumax', 428.66),
    ('Beta', 2.029),
    ('Pmax', 0.117),
    ('Tlag', 7.718),
    ('Kf', 0.038),
    ('Ks', 0.006),
    ('FM', 0.976)
]

In [59]:
ensemble = []

for counter in range(N): 
    ensemble.append(ewatercycle.models.HBVLocal(forcing=ERA5_forcing))
    config_file, _ = ensemble[counter].setup(
                            parameters = parameters[counter],
                            initial_storage=s_0,
        cfg_dir = HBV_model_path)
    ensemble[counter].initialize(config_file)

In [9]:
# Using a progress bar gives an indication of how long the process will take. This is particularly useful when N is chosen as a large number.
f = IntProgress(min=0, max=N)
display(f)

# The HBV model returns values in mm/day so in order to compare this to the observed data it must b converted to m^3/s
catchment_area_km2 = 334882.8

def mmday_to_m3s(Q_sim_mmday, area_km2):
    return (Q_sim_mmday * area_km2) / 86.4

# Array to store level of fit values
objectives = []

# Loop over ensemble members
for ensembleMember in ensemble:
    Q_m = []
    time = []
    while ensembleMember.time < ensembleMember.end_time:
        ensembleMember.update()
        discharge_this_timestep = ensembleMember.get_value("Q")
        Q_m.append(discharge_this_timestep[0])
        time.append(ensembleMember.time_as_datetime)

    # Create DataFrame for model results
    Q_m = mmday_to_m3s(np.array(Q_m), catchment_area_km2)
    discharge_dataframe = pd.DataFrame({'model output': Q_m}, index=pd.to_datetime(time))
    
    # Calculate the model of best-fit function
    objective_this_model = RMSE(discharge_dataframe['model output'], flow['Q'], start_calibration, end_calibration)
    objectives.append(objective_this_model)

    # Free up memory
    del Q_m, time, discharge_dataframe, objective_this_model
    f.value += 1

# Clean up models to save memory
for ensembleMember in ensemble:
    ensembleMember.finalize()

IntProgress(value=0, max=150)

In [60]:
# Loop over ensemble members
for ensembleMember in ensemble:
    Q_m = []
    time = []
    while ensembleMember.time < ensembleMember.end_time:
        ensembleMember.update()
        discharge_this_timestep = ensembleMember.get_value("Q")
        Q_m.append(discharge_this_timestep[0])
        time.append(ensembleMember.time_as_datetime)

    # Create DataFrame for model results
    Q_m = mmday_to_m3s(np.array(Q_m), catchment_area_km2)
    discharge_dataframe = pd.DataFrame({'model output': Q_m}, index=pd.to_datetime(time))
    
    # Calculate the model of best-fit function
    objective_this_model = KGE2(discharge_dataframe['model output'], flow['Q'], start_calibration, end_calibration)
    objectives.append(objective_this_model)

    # Free up memory
    del Q_m, time, discharge_dataframe, objective_this_model
    f.value += 1

# Clean up models to save memory
for ensembleMember in ensemble:
    ensembleMember.finalize()

In [10]:
# Showing which parameter set is the best fit
parameters_minimum_index = np.argmin(np.array(objectives))
if np.min(np.array(objectives)) == np.inf:
    print("No real parameter is chosen")

parameters_minimum = parameters[parameters_minimum_index]

print(f'The best parameters are: {list(zip(param_names, np.round(parameters_minimum, decimals=3)))}')

[
    6.344878250557591,
    0.8097953213402922,
    600.8990557273515,
    1.3818231002681736,
    0.16376673654421722,
    5.484373442035809,
    0.010601983130824688,
    0.005234796506050036,
    0.17402740162331534
]

The best parameters are: [('Imax', 6.345), ('Ce', 0.81), ('Sumax', 600.899), ('Beta', 1.382), ('Pmax', 0.164), 
('Tlag', 5.484), ('Kf', 0.011), ('Ks', 0.005), ('FM', 0.174)]

In [62]:
# Showing which parameter set is the best fit
parameters_maximum_index = np.argmax(np.array(objectives))
if np.min(np.array(objectives)) == np.inf:
    print("No real parameter is chosen")

parameters_maximum = parameters[parameters_maximum_index]
print(parameters_maximum)

print(f'The best parameters are: {list(zip(param_names, np.round(parameters_maximum, decimals=3)))}')

[5.94589424e+00 9.44340159e-01 3.62380790e+02 8.38720310e-01
 2.37515550e-01 5.75145126e+00 6.64406174e-02 5.61700659e-03
 1.38748689e-01]

The best parameters are: [('Imax', 5.946), ('Ce', 0.944), ('Sumax', 362.381), ('Beta', 0.839), ('Pmax', 0.238), 
('Tlag', 5.751), ('Kf', 0.066), ('Ks', 0.006), ('FM', 0.139)]